## Import Required Packages

In [ ]:
import numpy as np
from girth.synthetic import create_synthetic_irt_dichotomous, create_synthetic_irt_polytomous
from girth import twopl_mml, onepl_mml, pcm_mml, multidimensional_twopl_mml
from girth import ability_map
from girth import tag_missing_data, validate_estimation_options

## Show the processing options

In [ ]:
validate_estimation_options().keys()

## Run the 2PL Model

In [ ]:
# Simulation Parameters
n_items = 25
n_people = 500

# Create Synthetic Data
difficulty = np.linspace(-2.5, 2.5, n_items)
discrimination = np.random.rand(n_items) + 0.5
theta = np.random.randn(n_people)

syn_data = create_synthetic_irt_dichotomous(difficulty, discrimination, theta)

# Solve for parameters
results = twopl_mml(syn_data)

# Print all the keys in the results
print(results.keys())

## Compare the Results

In [ ]:
list(zip(discrimination.round(3), results['Discrimination'].round(3)))

## Run the 1PL Model

In [ ]:
# Simulation Parameters
n_items = 25
n_people = 500

# Create Synthetic Data
difficulty = np.linspace(-2.5, 2.5, n_items)
discrimination = 1.15
theta = np.random.randn(n_people)

syn_data = create_synthetic_irt_dichotomous(difficulty, discrimination, theta)

# Solve for parameters
results = onepl_mml(syn_data)

# Print all the keys in the results
print(results.keys())

# There isn't any ability in the dictionary, use the supplied functions
onepl_ability = ability_map(syn_data, results['Difficulty'], results['Discrimination'])

print(f"True vs Recovered Discrimination: {discrimination} : {results['Discrimination'].round(2)}")

## Note: 
The ```The rasch_mml``` and ```rasch_jml``` assume an underlying standardized normal distribution which is a constrained parameterization. This implementation was chosen so models are nested. The "traditional" Rasch model is the ```onepl_mml``` and ```onepl_jml``` methods. If one runs ```rasch_conditional``` then the standard deviation is reflected in the difficulty estimates

In [ ]:
from girth import rasch_conditional

results_conditional = rasch_conditional(syn_data)
conditional_ability = ability_map(syn_data, results_conditional['Difficulty'], results_conditional['Discrimination'])

print(results_conditional['Difficulty'].std(ddof=1)/ results['Difficulty'].std(ddof=1),
      results['Discrimination'])

## Generalized Partial Credit Polytomous Model

In [ ]:
# Simulation Parameters
n_items = 25
n_people = 500
n_levels = 3

# Create Synthetic Data
difficulty = np.random.randn(n_items, n_levels)
discrimination = np.random.rand(n_items) + 0.5
theta = np.random.randn(n_people)

syn_data = create_synthetic_irt_polytomous(difficulty, discrimination, theta, model='pcm')

# Solve for parameters
results = pcm_mml(syn_data)

# Print all the keys in the results
print(results.keys())


## Missing Data Example

In [ ]:
# Create Some missing data
percent_missing = 10
missing_mask = np.random.rand(*syn_data.shape) < (percent_missing / 100)

missing_data = syn_data.copy()
missing_data[missing_mask] = 29

## Tag the missing data to process
missing_data = tag_missing_data(missing_data, [1, 2, 3, 4])

# Solve for parameters
results_missing = pcm_mml(missing_data)

# Print all the keys in the results
print(results.keys())

## Print the difference between full data and missing data results

In [ ]:
list(zip(results['Discrimination'].round(3), results_missing['Discrimination'].round(3)))

## Multidimensional Example

In [ ]:
# Simulation Parameters
n_items = 25
n_people = 500
n_factors = 2

# Create Synthetic Data
difficulty = np.linspace(-1, 1, n_items)
discrimination = np.random.randn(n_items, n_factors) * .5
theta = np.random.randn(n_factors, n_people)

# Set a couple values to zeros just for comparison purposes
discrimination[-1, -1] = 0
discrimination[-1, 0] = np.abs(discrimination[-1, 0])
discrimination[-2, -1] = np.abs(discrimination[-2, -1])

syn_data = create_synthetic_irt_dichotomous(difficulty, discrimination, theta)

# Solve for parameters
results = multidimensional_twopl_mml(syn_data, n_factors, {'quadrature_n': 31})

# Print all the keys in the results
print(results.keys())


## Plot the difference in recovered values

In [ ]:
(results['Discrimination'] - discrimination).round(2)